# 01 — Keşifsel Veri Analizi (EDA)

Bu notebook'ta ham veriyi inceleyerek:
- Eksik değerleri tespit ediyoruz
- Hedef değişkenin (is_canceled) dağılımını anlıyoruz
- Önemli özelliklerin hedefle ilişkisini görselleştiriyoruz
- Encoding/feature engineering kararlarını temellendiriyoruz

In [ ]:
import sys

sys.path.insert(0, "..")

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

## 1. Veri Yükleme

In [ ]:
from src.io import read_input_dataset
from src.config import Paths

paths = Paths()
df, _ = read_input_dataset(paths.raw_data)
print(f"Shape: {df.shape}")
df.head()

## 2. Temel İstatistikler

In [ ]:
print("=== Dtypes ===")
print(df.dtypes.to_string())
print(f"\nEksik değer toplamı: {df.isnull().sum().sum()}")
df.describe().T

In [ ]:
# Eksik değer raporu
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if missing.empty:
    print("Eksik değer yok.")
else:
    (missing / len(df) * 100).plot(kind="bar", title="Eksik Değer Oranı (%)")
    plt.ylabel("%")
    plt.tight_layout()
    plt.show()

## 3. Hedef Değişken Dağılımı

In [ ]:
target_col = "is_canceled"
counts = df[target_col].value_counts()
print(counts)
print(f'İptal oranı: {counts.get(1, counts.get("yes", 0)) / len(df):.1%}')

fig, ax = plt.subplots()
counts.plot(kind="bar", ax=ax, color=["#2563eb", "#dc2626"], edgecolor="none")
ax.set_title("Hedef Dağılımı: is_canceled")
ax.set_xlabel("Sınıf")
ax.set_ylabel("Kayıt Sayısı")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{int(x):,}"))
plt.tight_layout()
plt.show()

## 4. Sayısal Değişkenler

In [ ]:
numeric_cols = df.select_dtypes(include="number").columns.tolist()
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

print(f"Sayısal kolon sayısı: {len(numeric_cols)}")
df[numeric_cols].hist(bins=30, figsize=(16, 8), edgecolor="none", color="#3b82f6")
plt.suptitle("Sayısal Özelliklerin Dağılımı", y=1.02)
plt.tight_layout()
plt.show()

## 5. Aylık İptal Trendi (Cyclic Encoding Gerekçesi)

In [ ]:
if "arrival_date_month" in df.columns:
    month_order = [
        "January",
        "February",
        "March",
        "April",
        "May",
        "June",
        "July",
        "August",
        "September",
        "October",
        "November",
        "December",
    ]
    cancel_by_month = (
        df.groupby("arrival_date_month")[target_col].mean().reindex(month_order)
    )

    fig, ax = plt.subplots()
    cancel_by_month.plot(kind="bar", ax=ax, color="#6366f1", edgecolor="none")
    ax.set_title(
        "Aylara Göre İptal Oranı\n(Ocak–Aralık sürekliliği cyclic encoding ile korunur)"
    )
    ax.set_ylabel("İptal Oranı")
    ax.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("arrival_date_month kolonu bulunamadı.")

## 6. Ülke Dağılımı (Frequency Encoding Gerekçesi)

In [ ]:
if "country" in df.columns:
    n_unique = df["country"].nunique()
    print(f"Benzersiz ülke sayısı: {n_unique} — OHE ile {n_unique} kolon üretilirdi.")
    print("Frequency encoding ile 1 kolon kullanılıyor (cardinal sıkıştırma).")

    top20 = df["country"].value_counts().head(20)
    fig, ax = plt.subplots(figsize=(12, 4))
    top20.plot(kind="bar", ax=ax, color="#0891b2", edgecolor="none")
    ax.set_title("İlk 20 Ülke (Rezervasyon Sayısı)")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("country kolonu bulunamadı.")

## Sonuç

| Gözlem | Karar |
|---|---|
| `arrival_date_month` string ay adı | Cyclic sin/cos encoding |
| `country` 100+ kategori | Frequency encoding |
| Eksik değer az/yok | Median imputer yeterli |
| Sınıf dengesizliği ~%37 iptal | `class_weight='balanced'` veya maliyet matrisi |

Bir sonraki notebook: `02_feature_engineering.ipynb`